<a href="https://colab.research.google.com/github/janamoumita1997/QA_system/blob/main/QA_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
!pip install datasets
import torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#Install python-docx
!pip install python-docx #<-- Yes you can directly install in Colab

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#Import the tools
import docx
from google.colab import files

uploaded = files.upload() #<-- Select the file you want to upload
file_name = '/content/sample_policy_document.docx' #<-- Change filename to your file
doc = docx.Document(file_name)

Saving sample_policy_document.docx to sample_policy_document.docx


In [ ]:
text = ''
for para in doc.paragraphs:
  # print(para.text)
  text += para.text
print("text >>>> ",text)

text >>>>  D.2.1 Total disability benefitIf the person insured is totally disabled, we will pay you the total disability benefit. The total disability benefit is calculated monthly and we will pay you half a month in arrears and half a month in advance. The definition of totally disabled depends on the person insured’s occupation category. Where the Schedule specifies the occupation category as MP, AA, A, B or C The person insured is totally disabled if, because of an injury or sickness, he or she is: not capable of doing the important duties of his or her occupation not working in any occupation (whether paid or unpaid), and under medical care. However, if immediately preceding a claim the person insured has been unemployed for 15 months or on leave without pay for 12 months, he or she is totally disabled if, because of an injury or sickness, he or she is: not capable of performing any occupation (whether paid or unpaid) for which he or she is reasonably suited by education, training 

In [ ]:
segmented_docs = [text]
segmented_docs

['D.2.1 Total disability benefitIf the person insured is totally disabled, we will pay you the total disability benefit. The total disability benefit is calculated monthly and we will pay you half a month in arrears and half a month in advance. The definition of totally disabled depends on the person insured’s occupation category. Where the Schedule specifies the occupation category as MP, AA, A, B or C The person insured is totally disabled if, because of an injury or sickness, he or she is: not capable of doing the important duties of his or her occupation not working in any occupation (whether paid or unpaid), and under medical care. However, if immediately preceding a claim the person insured has been unemployed for 15 months or on leave without pay for 12 months, he or she is totally disabled if, because of an injury or sickness, he or she is: not capable of performing any occupation (whether paid or unpaid) for which he or she is reasonably suited by education, training or experi

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def get_top_k_articles(query, docs, k=2):

  # Initialize a vectorizer that removes English stop words
  vectorizer = TfidfVectorizer(analyzer="word", stop_words='english')

  # Create a corpus of query and documents and convert to TFIDF vectors
  query_and_docs = [query] + docs
  matrix = vectorizer.fit_transform(query_and_docs)

  # Holds our cosine similarity scores
  scores = []

  # The first vector is our query text, so compute the similarity of our query against all document vectors
  for i in range(1, len(query_and_docs)):
    scores.append(cosine_similarity(matrix[0], matrix[i])[0][0])

  # Sort list of scores and return the top k highest scoring documents
  sorted_list = sorted(enumerate(scores), key=lambda x: x[1], reverse=True)
  top_doc_indices = [x[0] for x in sorted_list[:k]]
  top_docs = [docs[x] for x in top_doc_indices]
  
  return top_docs

In [ ]:
from transformers import BertTokenizer, BertForQuestionAnswering

model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

In [ ]:
def answer_question(question, answer_text):

    input_ids = tokenizer.encode(question, answer_text, max_length=512)
    
    # ======== Set Segment IDs ========
    # Search the input_ids for the first instance of the `[SEP]` token.
    sep_index = input_ids.index(tokenizer.sep_token_id)

    # The number of segment A tokens includes the [SEP] token istelf.
    num_seg_a = sep_index + 1

    # The remainder are segment B.
    num_seg_b = len(input_ids) - num_seg_a

    # Construct the list of 0s and 1s.
    segment_ids = [0]*num_seg_a + [1]*num_seg_b

    # There should be a segment_id for every input token.
    assert len(segment_ids) == len(input_ids)

    outputs = model(torch.tensor([input_ids]), # The tokens representing our input text.
                    token_type_ids=torch.tensor([segment_ids]), # The segment IDs to differentiate question from answer_text
                    return_dict=True) 

    start_scores = outputs.start_logits
    end_scores = outputs.end_logits

    # ======== Reconstruct Answer ========
    # Find the tokens with the highest `start` and `end` scores.
    answer_start = torch.argmax(start_scores)
    answer_end = torch.argmax(end_scores)

    # Get the string versions of the input tokens.
    tokens = tokenizer.convert_ids_to_tokens(input_ids)

    # Start with the first token.
    answer = tokens[answer_start]

    # Select the remaining answer tokens and join them with whitespace.
    for i in range(answer_start + 1, answer_end + 1):
        
        # If it's a subword token, then recombine it with the previous token.
        if tokens[i][0:2] == '##':
            answer += tokens[i][2:]
        
        # Otherwise, add a space then the token.
        else:
            answer += ' ' + tokens[i]

    print('Answer: "' + answer + '"')

In [ ]:
# Enter our query here
query1 = 'How often total disability is calculated? '
query2 = "How to pay the total disability?"
query3 = "What the definition of total disability depends on?"
query4 = "The definition of total disability for occupation category as MP, AA, A, B or C?"

queries = [query1, query2, query3, query4]

# Segment our documents
# segmented_docs = segment_documents(docs, 450)

# Retrieve the top k most relevant documents to the query
for query in queries:

  candidate_docs = get_top_k_articles(query, segmented_docs, 5)

  # Return the likeliest answers from each of our top k most relevant documents in descending order
  for i in candidate_docs:
    print('Query : ', query)
    answer_question(query, i)
    print('\n')

Query :  How often total disability is calculated? 
Answer: "monthly"


Query :  How to pay the total disability?
Answer: "the total disability benefit is calculated monthly"


Query :  What the definition of total disability depends on?
Answer: "the person insured ’ s occupation category"


Query :  The definition of total disability for occupation category as MP, AA, A, B or C?
Answer: "totally disabled"


